**EXERCISE-1**

---



Given optimization problem is:
$$ \text{max}\ 0.85x_1+3.75x_2-8.75x_3-3.45x_4-22.38x_5+20x_6 \\ \text{s.t.} -0.7x_1-3.9x_2-x_3+x_5\leq 14.9\\ x_1-x_4-x_6\leq0.9\\ -3x_2-2x_3-x_6\leq -51.7\\ 4.8x_4+9.12x_5+7.2x_6\leq 75 \\ -0.8x_1-1.5x_2+8x_3-2x_4+x_5\leq-27\\x_i\geq0 \  \forall \ i \in \{1,2,...,5\}
$$

In [25]:
!pip install -q pyomo

In [26]:
from pyomo.environ import *

In [27]:
import numpy as np

In [28]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter = ',')

In [29]:
print(coef.shape)
print('Number of rows:', coef.shape[0])
print('Number of columns:', coef.shape[1])

(6, 7)
Number of rows: 6
Number of columns: 7


In [30]:
print(coef)

[[  0.85   3.75  -8.75  -3.45 -22.38  20.     0.  ]
 [ -0.7   -3.9   -1.     0.     1.     0.    14.9 ]
 [  1.     0.     0.    -1.     0.    -1.     0.9 ]
 [  0.    -3.    -2.     0.     0.    -1.   -51.7 ]
 [  0.     0.     0.     4.8    9.12   7.2   75.  ]
 [ -0.8   -1.5    8.    -2.     1.     0.   -27.  ]]


In [31]:
model = ConcreteModel()

In [32]:
M = coef.shape[0] - 1 #number of constraints
N = coef.shape[1] - 1 #number of decision variables

In [33]:
print(M)
print(N)

5
6


In [34]:
obj_coef = coef[0,:-1]
constr_coef = coef[1:,:-1]
constr_rhs = coef[1:,-1]
lower_bound = 0
upper_bound = np.inf


In [35]:
row_indices = np.arange(M)
col_indices = np.arange(N)

In [36]:
print(col_indices)
print(row_indices)

[0 1 2 3 4 5]
[0 1 2 3 4]


In [37]:
model.x = Var(col_indices)

In [38]:
model.constraints = ConstraintList()

In [39]:
for i in row_indices:
      model.constraints.add(sum(constr_coef[i][j]*model.x[j] for j in col_indices)<= constr_rhs[i])

In [40]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [41]:
model.objective = Objective(expr = sum(obj_coef[j]*model.x[j] for j in col_indices), sense = maximize)

In [42]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [43]:
!apt-get install -y -qq coinor-cbc

In [44]:
opt_cbc = SolverFactory('cbc')

In [45]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>
Solver status: warning
Solver termination condition: unbounded


#Que.1(7)

The termination condition is unbounded, i.e., in this problem, we can maximize the objective function indefinitely without violating the constraints.

The solver status is warning.

We were solving the problem in the wrong optimization sense, so the feasible region is unbounded.

In [46]:
# reset the sense to minimize
model.objective.set_sense(minimize)
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [47]:
result = opt_cbc.solve(model)
print(result)
print('SolverStatus:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -97.02184211
  Upper bound: -97.02184211
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.025729656219482422
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

SolverStatus: ok
Termination Condition: optimal


**optimal value,value of the decision variables and value of the constraints:-**

In [48]:
print('Objective Function:', model.objective())
print('\nDecision Variables: ')
for j in col_indices:
      print('x[',j,']= ', model.x[j].value)
print('\nConstraints:')
model.constraints.display()      

Objective Function: -97.02184239599997

Decision Variables: 
x[ 0 ]=  0.9
x[ 1 ]=  23.002456
x[ 2 ]=  0.0
x[ 3 ]=  0.0
x[ 4 ]=  8.2236842
x[ 5 ]=  0.0

Constraints:
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


#Que.1(9)
Optimal Objective Value= -97.02184

value of the decision variables: $x_1 = 0.9\\ x_2 = 23.0024 \\ x_3 = 0 \\ x_4 = 0\\ x_5 = 8.2236842 \\ x_6 = 0$

Constraint 2, 4 and 5 (4 and 5 in approximate sense) are active since they attain exact or approximate equality.


After changing the sense from maximize to minimize, we are now getting an optimal solution. The previous termination condition was due to wrong sense of the optimization problem.